In [1]:
import torch
from dataset import SentimentAnalysisDataset
import pickle
from model import SentimentLSTM
from torch.nn import CrossEntropyLoss

In [1]:
import numpy as np
import torch
from dataset import SentimentAnalysisDataset

def load_embeddings(pretrained_file):
    embeddings = {}
    with open(pretrained_file,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embeddings[word] = np.array(split_line[1:], dtype=np.float64)
    print(f"{len(embeddings)} words loaded!")
    return embeddings

emb = load_embeddings('glove.6B.100d.txt')

In [4]:
emb_dim = len(emb['the'])
emb_dim

100

In [5]:
tokens = []
for t in ['the', 'cat', 'kwjelrk']:
    if t in emb:
        tokens.append(torch.from_numpy(emb[t]))
    else:
        tokens.append(torch.zeros(emb_dim))
tokens

[tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
         -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
          0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
          0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
          0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
         -0.7179, -0.4153,  0.2034, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
         -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9978, -0.8048, -3.0243,
          0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
          1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
         -0.3080, -0.4163,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
          0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
          0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
         -0.5203, -0.1459,  

In [7]:
X = torch.stack(tokens)
X.shape

torch.Size([3, 100])

In [8]:
dataset = SentimentAnalysisDataset('data/trainEmotions.csv', emb, embedding_dim=100)
tokens, emotion = dataset[0]
tokens.shape, emotion

/Users/snirlugassy/Projects/sentiment_analysis/dataset.py:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_croot-udngs7fm/pytorch_1648016055234/work/torch/csrc/utils/tensor_new.cpp:201.)
  for t in self.df.iloc[idx]['clean']:


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [3]:
dataset = SentimentAnalysisDataset('data/trainEmotions.csv', vocab)
tokens, emotion = dataset[0]

model = SentimentLSTM(len(vocab))

In [4]:
x = model.embedding(tokens)
x.shape

torch.Size([11, 300])

In [5]:
x,_ = model.lstm(model.embedding(tokens))
x.shape

torch.Size([11, 400])

In [6]:
output = model.fc(x)
output.shape, output

(torch.Size([11, 3]),
 tensor([[-0.1283,  0.0213,  0.0023],
         [-0.1297,  0.0250,  0.0052],
         [-0.1271,  0.0224,  0.0036],
         [-0.1268,  0.0215,  0.0012],
         [-0.1231,  0.0217,  0.0051],
         [-0.1247,  0.0250,  0.0030],
         [-0.1209,  0.0241,  0.0016],
         [-0.1177,  0.0232,  0.0018],
         [-0.1152,  0.0215,  0.0032],
         [-0.1181,  0.0205,  0.0060],
         [-0.1237,  0.0193,  0.0065]], grad_fn=<AddmmBackward0>))

In [14]:
output[-1].shape, emotion.shape

(torch.Size([3]), torch.Size([3]))

In [30]:
loss(output[-1].view(1,-1), emotion.view(1,-1))

tensor(1.1917, grad_fn=<DivBackward1>)

In [ ]:
torch.Tensor([emotion])